In [16]:
import numpy as np 
import nltk
import string 
import random
import os
import pandas as pd 
import re
import warnings
warnings.filterwarnings("ignore")

### Reading Data 

In [51]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...


True

In [87]:
nltk.download('punkt')  #download module for tokenization

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
data1 = open('Python.txt', 'r',encoding='utf-8')       #f = data1, #m = data2
data2 = open('modules.txt','r',encoding = 'utf-8')
checkpoint = "./chatbot_weights.ckpt"
raw_data1 = data1.read().lower()
raw_data2 = data2.read().lower()


### Preprocessing 

In [50]:
# tokenization
sent_tokens1 = nltk.sent_tokenize(raw_data1)
word_tokens1 = nltk.word_tokenize(raw_data1)
sent_tokens2 = nltk.sent_tokenize(raw_data2)
word_tokens2 = nltk.word_tokenize(raw_data2)

In [28]:
#lemmatization
from nltk.stem import WordNetLemmatizer
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punc_dict = dict((ord(punct),None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

Basic intro and QnA

In [151]:
Intro = ["My name is PyMentor.","I am PyMentor :) ","My name is PyMentor and I am happy to solve your queries :) ","Hi I am PyMentor, Ask me anything related to Python","Hi I am PyMentor, How can help you ?"]
greeting = ['hello',"hey","hi","hie","hellu","hiiii","hello there","hey there","hi there","wssup"]
greeting_response = ["hi","hello","hi there"]
Basic_Q1 = ("what is python ?","what is python","what is python?","what is python.")
Basic_Ans1 = "Python is an interpreted, object-oriented, high-level programming language with dynamic semantics. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured, object-oriented and functional programming."
Basic_Q2 = ("what is module","what is module.","what is module ","what is module ?","what is module?","what is module in python","what is module in python.","what is module in python?","what is module in python ?")
Basic_Ans2 = ["Modules in Python are files containing Python code, which can include variables, functions, and classes. They help organize code into reusable parts, making it easier to manage and maintain large projects. You can import modules into your Python programs to access their functionality.","Consider a module to be the same as a code library.","A file containing a set of functions you want to include in your application.","A module can define functions, classes and variables. A module can also include runnable code. Grouping related code into a module makes the code easier to understand and use."]
goodbye = ["Welcome!","Happy to help","My pleasure","I am glad I could help you!"]

helper functions

In [72]:
def greet(sentence):
    for word in sentence.split():
        if word.lower() in greeting:
            return random.choice(greeting_response)

def basic1(sentence):
    for word in Basic_Q1:
        if sentence.lower() == word:
            return Basic_Ans1
def basic2(sentence):
    for word in Basic_Q2:
        if sentence.lower() == word:
            return random.choice(Basic_Ans2)

def Introduction(sentence):
    return random.choice(Intro)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

response generation

In [107]:
#reponse generation 
def response1(user_response):
    bot_response = ''
    sent_tokens1.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
    tfidf = TfidfVec.fit_transform(sent_tokens1)
    vals = cosine_similarity(tfidf[-1],tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf == 0):
        bot_response = bot_response + "I'm sorry, I do not understand!"
        return bot_response
    else:
        bot_response = bot_response + sent_tokens1[idx]
        return bot_response



In [109]:
def response2(user_response):
    bot_response = ''
    sent_tokens2.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
    tfidf = TfidfVec.fit_transform(sent_tokens2)
    vals = cosine_similarity(tfidf[-1],tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf == 0):
        bot_response = bot_response + "I'm sorry, I do not understand!"
        return bot_response
    else:
        bot_response = bot_response + sent_tokens2[idx]
        return bot_response

In [156]:

def chat(user_response,flag):
    user_response = user_response.lower()
    keyword = "module "
    keywordone = " module" 
    keywordsec = " module "

    if(user_response!='bye'):
        if(user_response == 'thanks' or user_response == 'thank you'):
            flag = False
            print("PyMentor: ", random.choice(goodbye))
            #return random.choice("Welcome!","Happy to help","My pleasure","I am glad I could help you!")
            return flag
        elif(basic2(user_response)!=None):
            print("PyMentor: ",basic2(user_response))
            #return basic2(user_response)
            return flag
        else:
            if(user_response.find(keyword) != -1 or user_response.find(keywordone) != -1 or user_response.find(keywordsec)!=-1):
                print("PyMentor: ",response2(user_response))
                #return response2(user_response)
                sent_tokens2.remove(user_response)
                return flag
            elif(greet(user_response)!= None):
                print("PyMentor: ",greet(user_response))
                #return greet(user_response)
                return flag
            elif(user_response.find("your name") != -1 or user_response.find(" your name") != -1 or user_response.find("your name ") != -1 or user_response.find(" your name ") != -1): 
                print("PyMentor: ",Intro(user_response))
                #return Intro(user_response)
                return flag
            elif(basic1(user_response)!= None): 
                print("PyMentor: ",basic1(user_response))
                #return basic1(user_response)
                return flag
            else:
                print("PyMentor: ",response1(user_response))
                #return response1(user_response)
                sent_tokens1.remove(user_response)
                return flag
    else:
        flag = False
        print("PyMentor: Bye")
        #return "Bye! take care!"
        return flag
            

In [158]:
flag = True
while(flag == True):
    print("User: ", end = '')
    user_response = input()
    flag = chat(user_response,flag)

User: 

 thanks


PyMentor:  My pleasure
